In [2]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
Embedding_model = 'text-embedding-ada-002' 

text_splitter = CharacterTextSplitter(
    chunk_size=1000, chunk_overlap=50
)

paper_id = 1
paper_file =f'../data/papers/{paper_id}.pdf'
pages = PyPDFLoader(paper_file).load_and_split()

sliced_pages = text_splitter.split_documents(pages)
faiss_vectorstore = FAISS.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))
retriever = faiss_vectorstore.as_retriever(search_type='mmr',
        search_kwargs={"k": 20, "fetch_k": 10},
    )

llm=ChatOpenAI(
                model_name='gpt-4',
                temperature=0.1,
                )

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=retriever,
                                    memory=None, return_source_documents=False
                                )
        

/home/mehrad/anaconda3/envs/llm-hack/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
result = qa_chain({"query": "What are the design guidelines for making the MOF have a higher band gap?"})
result

/home/mehrad/anaconda3/envs/llm-hack/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'What are the design guidelines for making the MOF have a higher band gap?',
 'result': 'Based on the information provided, the band gap in Metal-Organic Frameworks (MOFs) can be influenced by several factors:\n\n1. Substituting the organic linker: The band gap can be significantly affected by exchanging the organic linker in the MOF structure. For instance, substituting hydrogen with NH2 or NO2 on the linker can reduce the band gap.\n\n2. Hydroxylation and Dehydroxylation: The band gap is larger for hydroxylated'}